In [32]:
import os
import sys

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory of `maps` (which is `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Add `src` to the module search path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [33]:
import pandas as pd
import numpy as np
import folium.features
from sklearn.cluster import DBSCAN
from sklearn.cluster import HDBSCAN
from sklearn.cluster import OPTICS
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures
from pyproj import Transformer
import seaborn as sns
import networkx as nx
from scipy.spatial import Delaunay
from itertools import combinations
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import pyproj


from python_scripts.road_detection.road_utils import *
from python_scripts.miscellaneaous.data_processing import extract_data
from python_scripts.graphs.graphs_creation import delaunay_graph
from python_scripts.city.city_utils import plotMapWithColors, mean_distance_choice, mean_distance_to_NN
from python_scripts.ihm.maps.mapUtils import *
from python_scripts.neighbours_criteria.enhanced_criteria import km_distance
from python_scripts.neighbours_criteria.simple_criteria import distance_criterion, angle_criterion


# Detect cities

In [34]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')

/tmp/ipykernel_16276/1038301328.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')


In [35]:
df_extracted = extract_data(df, provider='Orange')#, techno='4g')#, region='Normandie') #Occitanie, Île-de-France
# G, pos = delaunay_graph(df_extracted)

In [36]:
cityLabels = pd.Series(DBSCAN(eps=1000, min_samples=11).fit(df_extracted[['x','y']]).labels_, index = df_extracted.index)
cityLabels = cityLabels.drop(cityLabels.loc[cityLabels==-1].index)
clusters = cityLabels.unique()
num_clusters = len(clusters)

In [37]:
cityBsStationCount = cityLabels.value_counts()

In [38]:
def getCityCenters(df_extracted, clusters, cityBsStationCount):
    res = {cityLabel : (0,0) for cityLabel in clusters}
    for bs_id, x, y in df_extracted[['x', 'y']].loc[cityLabels.index].itertuples():
        coords = res.get(cityLabels[bs_id])
        res[cityLabels[bs_id]] = (x + coords[0], y + coords[1])

    for cluster in clusters:
        res[cluster] = res.get(cluster) / cityBsStationCount[cluster]
    return res

In [39]:
cityCenters = getCityCenters(df_extracted, clusters, cityBsStationCount)

In [40]:
# array1 = np.array(list(cityCenters.values())).reshape(-1,2)
# array2 = np.array(df_extracted[['x', 'y']].values)

# # Initialize the NearestNeighbors model and fit it to list2
# nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(array2)

# # Find the nearest neighbors for each point in list1
# distances, indices = nbrs.kneighbors(array1)

# Compute cities names
# cityNames = {cityId : df_extracted['nom_com'].loc[df_extracted.index[indice][0]] for (cityId, indice) in zip(cityCenters.keys(), indices)}

cityNames = {cityId : df_extracted['nom_com'].loc[cityLabels.loc[cityLabels==cityId].index].value_counts().index[0] for cityId in clusters}

# Merge clusters that have the same names
for cityName in np.sort(np.unique(list(cityNames.values()))):
    clustersWithSameName = [key for i, key in enumerate(list(cityNames.keys())) if list(cityNames.values())[i] == cityName]
    if(len(clustersWithSameName) > 1):
        clusterToKeep = np.min(clustersWithSameName)
        cityLabels = cityLabels.apply(lambda v: clusterToKeep if v in clustersWithSameName else v)
        for i in clustersWithSameName :
            if i != clusterToKeep:
                cityNames.pop(i)
clusters = list(cityLabels.unique())
num_clusters = len(clusters)
cityBsStationCount = cityLabels.value_counts()
cityCenters = getCityCenters(df_extracted, clusters, cityBsStationCount)

# Plot cities

In [41]:
def rgb_to_hex(rgb):
        return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))

def labelToColor(clustId, clusters, palette):   
    return rgb_to_hex(palette[list(clusters).index(clustId)])

In [42]:
lambert93 = pyproj.CRS("EPSG:2154")  # Lambert 93
wgs84 = pyproj.CRS("EPSG:4326")       # WGS84
transformer = pyproj.Transformer.from_crs(lambert93, wgs84, always_xy=True)

In [43]:
palette = sns.color_palette("hsv", num_clusters)
colors = cityLabels.apply(lambda clustId : labelToColor(clustId, clusters, palette))

map = folium.Map(location=np.mean(df_extracted[['latitude','longitude']], axis=0), zoom_start=7, tiles="Cartodb Positron")
citiesLayer = folium.FeatureGroup(f"Cities").add_to(map)
cityCenterLayer = folium.FeatureGroup(f"Cities centers").add_to(map)
for bs_id, latitude, longitude in df_extracted[['latitude', 'longitude']].loc[cityLabels.index].itertuples():
    color = colors[bs_id]
    dot = folium.CircleMarker(location=[latitude, longitude], color=color, radius=1, popup=cityNames.get(cityLabels[bs_id]))
    citiesLayer.add_child(dot)

for cityLabel in clusters:
    locationXY = cityCenters[cityLabel]
    locationLl = transformer.transform(locationXY[0], locationXY[1])[::-1]
    dot = folium.CircleMarker(location=locationLl, color='black', fill_opacity = 1, fill=True, radius=5, popup=cityNames.get(cityLabel))
    cityCenterLayer.add_child(dot)


folium.LayerControl().add_to(map)

map.save(f"../../out/maps/CitiesDetected.html")

/home/amandine/stage_CTU/lib/python3.12/site-packages/folium/utilities.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])


# Import Cities datas

In [44]:
cities_infos = pd.read_csv("../../database/data_population.csv", sep=";", decimal=',')

In [45]:
cities_infos.loc[cities_infos['Commune']=='Marseille']

,REG,Région,DEP,CODARR,CODCAN,CODCOM,COM,Commune,PMUN,PCAP,PTOT


In [46]:
biggest_cities = cities_infos['Commune'].loc[cities_infos['PMUN'].sort_values(ascending=False).index[:45]].values
print(biggest_cities)

np.sum([cityNames.get(cluster) in biggest_cities for cluster in clusters])/num_clusters

['Toulouse' 'Nice' 'Nantes' 'Montpellier' 'Strasbourg' 'Bordeaux' 'Lille'
 'Paris 15e Arrondissement' 'Rennes' 'Paris 20e Arrondissement'
 'Paris 18e Arrondissement' 'Paris 19e Arrondissement' 'Toulon' 'Reims'
 'Paris 13e Arrondissement' 'Saint-Étienne' 'Le Havre'
 'Paris 17e Arrondissement' 'Paris 16e Arrondissement' 'Dijon' 'Grenoble'
 'Angers' 'Villeurbanne' 'Saint-Denis' 'Nîmes' 'Aix-en-Provence'
 'Clermont-Ferrand' 'Le Mans' 'Paris 11e Arrondissement'
 'Paris 12e Arrondissement' 'Brest' 'Tours' 'Paris 14e Arrondissement'
 'Amiens' 'Annecy' 'Limoges' 'Metz' 'Boulogne-Billancourt' 'Perpignan'
 'Besançon' 'Orléans' 'Rouen' 'Saint-Denis' 'Montreuil' 'Caen']


np.float64(0.5714285714285714)

In [51]:
populations = {}

for cluster in clusters:
    cityName = cityNames.get(cluster)
    city_row = cities_infos[cities_infos['Commune']==cityName]
    population = city_row['PMUN'].values[0]
    populations[cluster]=population

print(f"Coefficient de corrélation : {np.corrcoef(cityBsStationCount.loc[cityNames.keys()].values, [populations.get(key) for key in cityNames.keys()])[0,1]}")

plt.figure(figsize=(15,10))
ax = plt.subplot(111)

X_axis = np.arange(len(cityNames.values())) 

nbBaseStationScores = cityBsStationCount.loc[cityNames.keys()].values/np.max(cityBsStationCount.values)
plt.bar(X_axis + 0.2, nbBaseStationScores, 0.4, label = 'NbBaseStion') 

populationScores = [populations.get(key) for key in cityNames.keys()]/np.max(list(populations.values()))
plt.bar(X_axis - 0.2, populationScores, 0.4, label = 'Population') 
  
plt.xticks(X_axis, cityNames.values()) 
plt.xlabel("Cities detected") 
plt.ylabel("% of the max") 
plt.title("Population vs nb of base station") 
plt.legend() 
plt.xticks(rotation=90)

plt.show() 



plt.show()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [48]:
representation_score = pd.Series(data = nbBaseStationScores / populationScores, index=cityNames.values()).sort_values(ascending=False)
print(representation_score)
print(np.median(representation_score))

Le Bourget           1.601214
Lille                1.058030
Paris                1.000000
Roubaix              0.851452
Nogent-sur-Marne     0.774027
Rennes               0.729006
Lyon                 0.715194
Villeneuve-d'Ascq    0.695706
Tourcoing            0.654176
Toulouse             0.548237
Cannes               0.530508
Nancy                0.496993
Orléans              0.370275
Bordeaux             0.362855
Montpellier          0.349779
Rouen                0.340650
Marseille            0.321475
Nice                 0.310128
Metz                 0.303649
Strasbourg           0.266808
Clermont-Ferrand     0.263783
Avignon              0.262916
Créteil              0.255398
Limoges              0.249578
Nantes               0.200402
Aix-en-Provence      0.161036
Nîmes                0.160355
Reims                0.156468
dtype: float64
0.3563167247554747
